# Calculate evolutionary rate by PIC

In [42]:
# data source: phylogenetic topology and current states of body weight and maximum longevity
# https://onlinelibrary.wiley.com/doi/epdf/10.1111/j.1558-5646.1991.tb04375.x

## 1. Load library

In [67]:
import toytree
import math

## 2. Load tree with branch length

In [68]:
# The unit here is MY
tre = toytree.tree("((((Homo:0.21,Pongo:0.21):0.28,Macaca:0.49):0.13,Ateles:0.62):0.38,Galago:1.00);")
tre.draw(ts = 'c', layout = 'r', node_labels = True, node_sizes = 15);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="280.0px" height="275.0px" viewBox="0 0 280.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc7927fb56edd43a4a751fc78f5967f1f"> 0 1 2 3 4 5 6 7 8 Homo Pongo Macaca Ateles Galago 1 0.5 0

## 3. Set current states

In [69]:
#set current body_weight (ln transformed)
tre = tre.set_node_data(feature = "body_weight", 
                  mapping = {0:math.log(60), 1:math.log(37), 2:math.log(10.7), 3:math.log(7.6), 4:math.log(0.23)}, default = None)

#set current max longevity
tre = tre.set_node_data(feature = "max_longevity", 
                  mapping = {0:math.log(115), 1:math.log(28), 2:math.log(29), 3:math.log(18), 4:math.log(10)}, default = None)

#display all node data
tre.get_node_data()

,idx,name,height,dist,support,body_weight,max_longevity
0,0,Homo,0.0,0.21,0.0,4.094345,4.744932
1,1,Pongo,0.0,0.21,0.0,3.610918,3.332205
2,2,Macaca,0.0,0.49,0.0,2.370244,3.367296
3,3,Ateles,0.0,0.62,0.0,2.028148,2.890372
4,4,Galago,0.0,1.0,0.0,-1.469676,2.302585
5,5,,0.21,0.28,0.0,,
6,6,,0.49,0.13,0.0,,
7,7,,0.62,0.38,0.0,,
8,8,,1.0,1.0,0.0,,


## 4. Traverse the tree from tip to root and assign state to internal nodes

In [70]:
def standard_contrast(node, attribute:str):
    '''
    Calculate the standard contrast which is the raw contrast divided by the sum of children branch lengths
    '''
    vi = node.children[0].dist
    vj = node.children[1].dist
    variance = vi + vj
    
    if attribute == 'body_weight':
        xi = node.children[0].body_weight
        xj = node.children[1].body_weight
        raw_contrast = xi - xj
    elif attribute == 'max_longevity':
        xi = node.children[0].max_longevity
        xj = node.children[1].max_longevity
        raw_contrast = xi - xj
    else:
        print(f"Check the attribute spelling. No {attribute} has been assigned to nodes")
        
    return raw_contrast/math.sqrt(variance)

In [71]:
def character_value(node,attribute:str):
    '''
    Calculate the character value of internal nodes which is the weighted average of its children node character value
    '''
    vi = node.children[0].dist
    vj = node.children[1].dist
    
    if attribute == 'body_weight':
        xi = node.children[0].body_weight
        xj = node.children[1].body_weight
        xk = ((1/vi)*xi+(1/vj)*xj)/((1/vi)+(1/vj))
    elif attribute == 'max_longevity':
        xi = node.children[0].max_longevity
        xj = node.children[1].max_longevity
        xk = ((1/vi)*xi+(1/vj)*xj)/((1/vi)+(1/vj))
    else:
        print(f"Check the attribute spelling. No {attribute} has been assigned to nodes")
        
    return xk

In [72]:
def lengthen_branch_length(node):
    '''
    Calculate lengthed branch lenght of internal nodes
    '''
    original = node.dist
    vi = node.children[0].dist
    vj = node.children[1].dist
    var = (vi*vj)/(vi+vj)
    
    return original+var

In [73]:
# copy tree
ntre = tre.copy()

# traverse the tree from tips to roots and calculate internal nodes values
for node in ntre.treenode.traverse("idxorder"):
    count = 0
    sum_body_weight = 0
    sum_max_longevity = 0
    if not node.is_leaf():
        # calculate standard contrast
        s_body_weight = standard_contrast(node, attribute = "body_weight")
        s_max_longevity = standard_contrast(node, attribute = "max_longevity")
        
        # calculate weighted value of two children nodes
        node.body_weight = character_value(node,attribute = "body_weight")
        node.max_longevity = character_value(node,attribute = "max_longevity")
        
        # elongate branch length
        node._dist = lengthen_branch_length(node)
        
        # calculate sum of standard contrasts
        sum_body_weight += s_body_weight**2
        sum_max_longevity += s_max_longevity**2
        count +=1

# calculate estimated evolutionary rate
evo_rate_body_weight = sum_body_weight/count
evo_rate_max_longevity = sum_max_longevity/count

print(f"The estimated evolutionary rate of body weight is: {evo_rate_body_weight}")
print(f"The estimated evolutionary rate of maximum longevity is: {evo_rate_max_longevity}")

# compare two trees
print(tre.get_node_data())
ntre.get_node_data()

The estimated evolutionary rate of body weight is: 11.278289061636565
The estimated evolutionary rate of maximum longevity is: 0.8047252712143309
  idx    name height  dist support body_weight max_longevity
0   0    Homo    0.0  0.21     0.0    4.094345      4.744932
1   1   Pongo    0.0  0.21     0.0    3.610918      3.332205
2   2  Macaca    0.0  0.49     0.0    2.370244      3.367296
3   3  Ateles    0.0  0.62     0.0    2.028148      2.890372
4   4  Galago    0.0   1.0     0.0   -1.469676      2.302585
5   5           0.21  0.28     0.0                          
6   6           0.49  0.13     0.0                          
7   7           0.62  0.38     0.0                          
8   8            1.0   1.0     0.0                          


,idx,name,height,dist,support,body_weight,max_longevity
0,0,Homo,0.0,0.21,0.0,4.094345,4.744932
1,1,Pongo,0.0,0.21,0.0,3.610918,3.332205
2,2,Macaca,0.0,0.49,0.0,2.370244,3.367296
3,3,Ateles,0.0,0.62,0.0,2.028148,2.890372
4,4,Galago,0.0,1.0,0.0,-1.469676,2.302585
5,5,,0.21,0.385,0.0,3.852631,4.038568
6,6,,0.49,0.3456,0.0,3.200381,3.743208
7,7,,0.62,0.601906,0.0,2.780824,3.437968
8,8,,1.0,1.375743,0.0,1.183727,3.011355


## 5. Validation by comparing PIC values in R scripts

In [74]:
# calculate the standard character(PIC) of internal nodes
s_body_weight = []
s_max_longevity = []

for node in ntre.treenode.traverse("idxorder"):
    if not node.is_leaf():
        # calculate standard contrast
        s_body_weight.append(standard_contrast(node, attribute = "body_weight"))
        s_max_longevity.append (standard_contrast(node, attribute = "max_longevity"))

print(f"PIC of body weight:\n{s_body_weight}\n")
print(f"PIC of maximum longevity:\n{s_max_longevity}")   

PIC of body weight:
[0.7459435149081659, 1.5847388926685422, 1.1929304559816334, 3.3583164028477968]

PIC of maximum longevity:
[2.179886040703827, 0.7176204765894122, 0.8678950975774259, 0.8970648088150214]


In [27]:
# compare it to the result of a R script
# source: https://rdrr.io/cran/ape/man/pic.html
# output:
# PIC of body weight
#        6         7         8         9 
#3.3583189 1.1929263 1.5847416 0.7459333 
# PIC of maximum longevity
#        6         7         8         9 
#0.8970604 0.8678969 0.7176125 2.1798897